In [1380]:
import importlib
import utils
import transformers
import model
import losses

In [1381]:
from model import *
from utils import *

In [1382]:
importlib.reload(utils)
importlib.reload(transformers)
importlib.reload(model)
importlib.reload(losses)

<module 'losses' from 'd:\\Documents\\machinelearning\\cgm-paper\\src\\losses.py'>

In [1383]:
print("Current working directory: ", os.getcwd())

Current working directory:  d:\Documents\machinelearning\cgm-paper\src


In [1384]:
raw_dir = "../data/raw"
tuning_dir = "../data/tuning"

In [1385]:
assert os.path.exists(raw_dir), "Raw data directory not found"
assert os.path.exists(tuning_dir), "Tuning data directory not found"

In [1386]:
dataset = load_data(raw_dir)

Found 22 files in ../data/raw
train (139415, 47)
test (32912, 47)


In [1387]:
val_dataset = load_data(tuning_dir)

Found 2 files in ../data/tuning
train (13630, 43)
test (3002, 43)


In [1388]:
for key in dataset.keys():
    val_dataset[key] = reduce_classes(val_dataset[key])
    dataset[key] = reduce_classes(dataset[key])

In [1389]:
dataset.get("train")['Time'].isnull().sum()

0

In [1390]:
train_data = get_train_dataset(dataset['train'])
test_data = get_any_dataset(dataset['test'], dataset['train'])

      CGM  month  day  hour  minute  weekday  year  cgm_velo  change  \
24  129.0      5   19    13      35        2  2027  0.003333     1.0   
25  127.0      5   19    13      40        2  2027  0.000000     0.0   
26  124.0      5   19    13      45        2  2027 -0.006667    -2.0   
27  123.0      5   19    13      50        2  2027 -0.010000    -3.0   
28  124.0      5   19    13      55        2  2027 -0.003333    -1.0   

    upper_band  ...  extreme_CGM_80  extreme_CGM_85  extreme_CGM_90  \
24  198.092207  ...               0               0               0   
25  198.840303  ...               0               0               0   
26  198.288080  ...               0               0               0   
27  196.078435  ...               0               0               0   
28  192.197842  ...               0               0               0   

    extreme_CGM_95  wCGM(1)     wCGM(2)     wCGM(3)  wCGM(4)     wCGM(5)  \
24               0    129.0  128.333333  127.666667    127.8  12

In [1391]:
train_data

,CGM,month,day,hour,minute,weekday,year,cgm_velo,change,upper_band,...,extreme_CGM_80,extreme_CGM_85,extreme_CGM_90,extreme_CGM_95,wCGM(1),wCGM(2),wCGM(3),wCGM(4),wCGM(5),wCGM(6)
24,129.0,5,19,13,35,2,2027,0.003333,1.0,198.092207,...,0,0,0,0,129.0,128.333333,127.666667,127.8,126.866667,129.476190
25,127.0,5,19,13,40,2,2027,0.000000,0.0,198.840303,...,0,0,0,0,129.0,129.000000,128.500000,127.9,127.933333,127.095238
26,124.0,5,19,13,45,2,2027,-0.006667,-2.0,198.288080,...,0,0,0,0,127.0,128.333333,128.666667,128.4,127.933333,127.952381
27,123.0,5,19,13,50,2,2027,-0.010000,-3.0,196.078435,...,0,0,0,0,124.0,126.000000,127.500000,128.1,128.066667,127.761905
28,124.0,5,19,13,55,2,2027,-0.003333,-1.0,192.197842,...,0,0,0,0,123.0,123.666667,125.333333,126.8,127.533333,127.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139410,266.0,1,13,23,40,3,2022,0.076667,23.0,286.553226,...,1,1,1,0,247.0,231.666667,215.333333,207.6,205.733333,207.238095
139411,275.0,1,13,23,45,3,2022,0.063333,19.0,291.227760,...,1,1,1,1,266.0,253.333333,238.666667,222.8,213.866667,210.476190
139412,268.0,1,13,23,50,3,2022,0.030000,9.0,298.633314,...,1,1,1,0,275.0,269.000000,258.000000,244.4,229.266667,219.761905
139413,301.0,1,13,23,55,3,2022,-0.023333,-7.0,303.490145,...,1,1,1,1,268.0,272.666667,269.333333,260.4,248.266667,234.190476


The tuning data will be merged with the training data and be fitted into the pipeline to be used for the validation data

In [1392]:
tuning_data = get_tuning_dataset(val_dataset['train'], dataset['train'])
val_data = get_any_dataset(val_dataset['test'], tuning_data)

In [1393]:
trainX, trainY = train_data.drop(["CGM"], axis = 1), train_data["CGM"]
trainX.shape, trainY.shape

((120010, 20), (120010,))

In [1394]:
testX, testY = test_data.drop(["CGM"], axis = 1), test_data["CGM"]
testX.shape, testY.shape

((26845, 20), (26845,))

In [1395]:
model = HybridModel()
model.fit(trainX, trainY, testX, testY, eval = True, tune = False)

Base fitted with columns: Index(['month', 'day', 'hour', 'minute', 'weekday', 'year', 'cgm_velo',
       'change', 'upper_band', 'lower_band', 'extreme_CGM_80',
       'extreme_CGM_85', 'extreme_CGM_90', 'extreme_CGM_95', 'wCGM(1)',
       'wCGM(2)', 'wCGM(3)', 'wCGM(4)', 'wCGM(5)', 'wCGM(6)', 'direction'],
      dtype='object')
Base: (16.233020280604528, 4.029022248710539, 0.07677582969463718)
Base + Residuals: (16.147935747935648, 4.01844942085074, 0.07643198893322938) Change: 4.2914389644083%
